In [36]:
import tensorflow as tf
import numpy as np
import os


In [8]:
Null_TAG = 'None'
P_TAG_b = 'Pb'  # beginning of participant phrase
P_TAG_m = 'Pm'  # middle/end of participant phrase

In [7]:
abs_path = 'PICO-annotations/batch5k/0074f5e102cf4409ac07f6209dd30144/20957980.txt'
ann_path = 'PICO-annotations/batch5k/0074f5e102cf4409ac07f6209dd30144/20957980_gold.ann'

abs_file = open(abs_path, 'r');
file_text = abs_file.read();

print file_text

ann_file = open(ann_path, 'r');
ann_file = ann_file.read();

ann_list = ann_file.split();
part_list = [];
for i in range(1, len(ann_list), 2):
    part_list.append((int(ann_list[i]), int(ann_list[i+1])))

for participant_indices in part_list:
    start = participant_indices[0]
    end = participant_indices[1]
    print start
    print end
    print file_text[start:end]
    

Ipilimumab efficacy and safety in patients with advanced melanoma: a retrospective analysis of HLA subtype from four trials.

Ipilimumab is a fully human, monoclonal antibody that blocks cytotoxic T-lymphocyte antigen-4 to potentiate an antitumor T-cell response. This agent improved overall survival in a phase III trial in previously treated patients with advanced melanoma. Because the mechanism of action for ipilimumab is thought to be HLA independent, most trials enrolled patients without regard to HLA subtype. However, enrollment in the phase III trial was restricted to class-I HLA-A*0201-positive patients because two of the three arms contained an HLA-A*0201-restricted gp100 vaccine. HLA typing was also performed prospectively in several phase II trials and was available for 93.5% of patients. In this retrospective analysis, pooled efficacy and safety data are presented according to HLA-A*0201 status and dose from pretreated patients randomized to 0.3, 3, or 10 mg/kg ipilimumab in 

In [46]:
def annotate_abstract(abstract_path, gold_annotation_path):
    abs_file = open(abstract_path, 'r');
    file_text = abs_file.read();

    ann_file = open(gold_annotation_path, 'r');
    ann_file = ann_file.read();

    ann_list = ann_file.split();
    part_list = [];
    for i in range(1, len(ann_list), 2):
        part_list.append((int(ann_list[i]), int(ann_list[i+1])))


    word_list = file_text.split(); # [word1, word2, word] no spaces
    tag_list = []
    index = 0; # changed from -1
    ann_index = 0
    if (len(part_list) == 0):
        ann_start = np.inf
        ann_end = np.inf
    else:
        ann_start = part_list[ann_index][0]
        ann_end = part_list[ann_index][1]
    in_phrase = False
    sentence_start_ind = [0]

    for word_ind in range(len(word_list)):
        word = word_list[word_ind]
        index += len(word) + 1;
        if not in_phrase:
            # looking for start of participant phrase
            if (ann_start < index):
                # we found first word in this participant segment
                tag_list.append(P_TAG_b)
                in_phrase = True
            else:
                tag_list.append(Null_TAG)          
        else:
            tag_list.append(P_TAG_m)
            # in the participant phrase, looking for its end
            if (ann_end <= index):
                # we found the last word in the participant segment
                ann_index += 1
                if (ann_index == len(part_list)):
                    ann_start = np.inf
                    ann_end = np.inf
                else:
                    ann_start = part_list[ann_index][0]
                    ann_end = part_list[ann_index][1]
                in_phrase = False

        if word[-1] == ".":
            sentence_start_ind.append(word_ind+1)
                

    out_abs_path = abstract_path[0:-4] + '_input.txt'
    out_ann_path = abstract_path[0:-4] + '_input_tags.ann'
    
    abs_out_str = ''
    ann_out_str = ''
    
    for i in range(0, len(sentence_start_ind)-1):        
        sentence = ' '.join(word_list[sentence_start_ind[i]:sentence_start_ind[i+1]])
        tag_sentence = ' '.join(tag_list[sentence_start_ind[i]:sentence_start_ind[i+1]])
        
        abs_out_str = abs_out_str + sentence + '\n'
        ann_out_str = ann_out_str + tag_sentence + '\n'
    
    abs_f = open(out_abs_path, 'w')
    ann_f = open(out_ann_path, 'w')
    
    abs_f.write(abs_out_str);
    ann_f.write(ann_out_str);
    
    abs_f.close();
    ann_f.close();
    

In [43]:
abs_path = 'PICO-annotations/batch5k/0074f5e102cf4409ac07f6209dd30144/20957980.txt'
ann_path = 'PICO-annotations/batch5k/0074f5e102cf4409ac07f6209dd30144/20957980_gold.ann'
annotate_abstract(abs_path, ann_path)

In [47]:
directory = 'PICO-annotations/batch5k'

# For each subdirectory
for subdir in os.listdir(directory):
    subdir_path = directory + '/' + subdir
    print subdir_path
    
    # Not a directory
    if not os.path.isdir(subdir_path):
        continue
    
    # For each abstract in subdirectory
    for abstract in os.listdir(subdir_path):
        if (abstract.endswith('.txt')) and not (abstract.endswith('input.txt')):
            abstract_path = subdir_path + '/' + abstract; 
            print abstract_path
            ann_path = abstract_path[0:-4] + '_gold.ann'
            annotate_abstract(abstract_path, ann_path)

PICO-annotations/batch5k/0074f5e102cf4409ac07f6209dd30144
PICO-annotations/batch5k/0074f5e102cf4409ac07f6209dd30144/20957980.txt
PICO-annotations/batch5k/0074f5e102cf4409ac07f6209dd30144/23127528.txt
PICO-annotations/batch5k/0074f5e102cf4409ac07f6209dd30144/9665186.txt
PICO-annotations/batch5k/00a340c811a841f3bbf15f4828fd43dd
PICO-annotations/batch5k/00a340c811a841f3bbf15f4828fd43dd/2182235.txt
PICO-annotations/batch5k/00a340c811a841f3bbf15f4828fd43dd/23773944.txt
PICO-annotations/batch5k/00a340c811a841f3bbf15f4828fd43dd/9732932.txt
PICO-annotations/batch5k/00f9eddfa8674d2e86b2f5c065685de0
PICO-annotations/batch5k/00f9eddfa8674d2e86b2f5c065685de0/11004058.txt
PICO-annotations/batch5k/00f9eddfa8674d2e86b2f5c065685de0/11246578.txt
PICO-annotations/batch5k/00f9eddfa8674d2e86b2f5c065685de0/21295919.txt
PICO-annotations/batch5k/0153faa6ed8f438ca815dd9827262a65
PICO-annotations/batch5k/0153faa6ed8f438ca815dd9827262a65/1715416.txt
PICO-annotations/batch5k/0153faa6ed8f438ca815dd9827262a65/1790

In [ ]:

# CURRENTLY ASSUMES: 
def read_file(abstract_path, gold_annotation_path=None):    
    abstract_file = open(abstract_path, 'r');
    file_text = f.readlines();
    
    
    line_ids = file_text[0::2]
    sentances = file_text[1::2]

    split_s = np.array([line.replace('_', ' ').split() for line in sentances])

    words = np.array([arr[0::2] for arr in split_s])
    tags = np.array([arr[1::2] for arr in split_s])
    
    return [line_ids, words, tags]